# OECD renewable energy feed-in tariffs

The Organization for Economic Co-operation and Development ([OECD](https://www.oecd.org/)) publishes data about the [renewable energy feed-in tariffs](https://stats.oecd.org/Index.aspx?DataSetCode=RE_FIT) (FITs): 

> Feed-in tariffs (FITs) are prevalent support policies for scaling up renewable electricity capacity. They are market-based economic instruments, which typically offer long-term contracts that guarantee a price to be paid to a producer of a pre-determined source of electricity per kWh fed into the electricity grid.
>
> This dataset provides FITs values derived in a manner that is comparable across countries, years and renewable energy sub-sectors. The data include country-level values on the tariff (in USD/kWh), and length of the awarded power-purchasing agreement. The dataset covers seven renewable electricity sub-sectors: wind, solar photovoltaic (concentrated solar power is excluded), geothermal, small hydro, geothermal, marine, biomass and waste.
>
> This dataset is fully maintained by the OECD secretariat, drawing on government sources (e.g. websites, official documents) and information from research institutes. In addition, the data are cross-checked against other renewable energy policy databases (e.g. REN21, IEA/IRENA, [OECD PINE database](http://www.oecd.org/environment/indicators-modelling-outlooks/policy-instrument-database/)).
>
> For further details on the methodology and examples of applications of this dataset, please consult [the documentation](https://stats.oecd.org/wbos/fileview2.aspx?IDFile=7e7f7564-1046-4932-bfad-d24f2a679f15).


In [ ]:
import os
import pandas as pd
import datetime as dt
import pandas_datareader.data as web

In [ ]:
start_time = dt.datetime(2000, 1, 1)
end_time = dt.datetime(2022, 10, 1)
df = web.DataReader('RE_FIT', 'oecd', start_time, end_time)
df.head()

In [ ]:
# Move Country and Variable to rows
df = df.stack(level=[0, 1]).reset_index()
df.head()

The dataset contains 2 variables:
- "Length of power purchase agreement"
- "Mean feed-in tariff"

Keep only `Mean feed-in tariff`

In [ ]:
# Keep only the FIT variable
df = df[df["Variable"]=="Mean feed-in tariff"]
df.head()

In [ ]:
df = df.drop("Variable", axis=1)
df.head()

In [ ]:
# Keep only the year in the Year column
df["Year"] = [x.year for x in df["Year"]]
df.head()

In [ ]:
# Rename year and country. Remove spaces from column names
df.rename({"Year": "year", "Country": "country", "Small Hydro": "Small_Hydro", "Solar PV": "Solar_PV"}, axis=1, inplace=True)

In [ ]:
metadata_dir = "metadata"
path_to_country_conversion_table = os.path.join(metadata_dir, "country_name_conversion.csv")
if not os.path.exists(path_to_country_conversion_table):
  # get world bank country names and iso alpha3 code
  countries_table = pd.read_csv('../processed/ccdr.csv')
  countries_table = countries_table[['country_code','country_name']].drop_duplicates().reset_index(drop=True)
  countries_table.columns = ['country_code',	'wb_country_name']

  # get oecd country names
  # oecd_country_names = pd.read_csv('processed/fit.csv')
  oecd_country_names = list(df.country.unique())

  # match each oecd country name w/ wb country names
  # and store the oecd country name that do not match
  missing_countries = []
  for country in oecd_country_names:
    c = countries_table.loc[countries_table.wb_country_name== country,:]
    if len(c)==1:
      countries_table.loc[countries_table.wb_country_name== country,'oecd_country_name'] = country
    else:
      missing_countries.append(country)
elif len(missing_countries)>=0:
    print(f'WB and OECD country names do not match. Update table with missing {missing_countries}')
else:
  missing_countries =0
  countries_table = pd.read_csv(path_to_country_conversion_table)

In [ ]:
# manual update: check & verify each country name correspondence to ISO country codes
country_manual_update = {"China (People's Republic of)": 'CHN',
                         'Egypt': 'EGY',
                         'Iran':'IRN',
                         'Korea':'KOR', 
                         'Russia':'RUS', 
                         'Türkiye':'TUR'
                         }

In [ ]:
# Update country names in the country_name_conversion table
if len(missing_countries)>0:
  # update missing_country_codes
  for country in country_manual_update.items():
    countries_table.loc[countries_table.country_code==country[1],'oecd_country_name'] = country[0]

In [ ]:
df = pd.merge(df,
              countries_table.dropna(axis=0, how='any'),
              how='left',
              left_on=['country'],
              right_on=['oecd_country_name']
              ).drop(['wb_country_name','oecd_country_name'], axis=1)

In [ ]:
df.head(69)

## Check

In [ ]:
df[(df["year"] == 2011) & 
   (df["country"] == "Canada")]

In [ ]:
# Note: not the same countries for different years
df[df["year"] == 2000].shape

In [ ]:
df[df["year"] == 2019].shape

## Export 
Export to .csv file

In [ ]:
processed_dir = "processed"
csv_full_filename = os.path.join(processed_dir, "fit.csv")
csv_full_filename

In [ ]:
# Create the output directory if needed
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)
df.to_csv(csv_full_filename, encoding='utf-8', index=False)

In [ ]:
# Update country_name_conversion table
# Create the output directory if needed
if not os.path.exists(metadata_dir):
    os.makedirs(metadata_dir)

countries_table.to_csv(path_to_country_conversion_table, encoding='utf-8', index=False)